### baysian optimization

#### resources
* https://gpflowopt.readthedocs.io/en/latest/notebooks/constrained_bo.html baysian optimization with constraints.



In [50]:
test = False

In [52]:
# example of bayesian optimization for a 1d function from scratch
from math import sin
from math import pi
from numpy import arange
from numpy import vstack
from numpy import argmax
from numpy import asarray
from numpy.random import normal
from numpy.random import random
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from warnings import catch_warnings
from warnings import simplefilter
from matplotlib import pyplot

# objective function
def objective(x, noise=0.1):
    noise = normal(loc=0, scale=noise)
    return (x**2 * sin(5 * pi * x)**6.0) + noise

# surrogate or approximation for the objective function
def surrogate(model, X):
    # catch any warning generated when making a prediction
    with catch_warnings():
        # ignore generated warnings
        simplefilter("ignore")
        return model.predict(X, return_std=True)

# probability of improvement acquisition function
def acquisition(X, Xsamples, model):
    # calculate the best surrogate score found so far
    yhat, _ = surrogate(model, X)
    best = max(yhat)
    # calculate mean and stdev via surrogate function
    mu, std = surrogate(model, Xsamples)
    mu = mu[:, 0]
    # calculate the probability of improvement
    probs = norm.cdf((mu - best) / (std+1E-9))
    return probs

# optimize the acquisition function
def opt_acquisition(X, y, model):
    # random search, generate random samples
    Xsamples = random(100)
    Xsamples = Xsamples.reshape(len(Xsamples), 1)
    # calculate the acquisition function for each sample
    scores = acquisition(X, Xsamples, model)
    # locate the index of the largest scores
    ix = argmax(scores)
    return Xsamples[ix, 0]

# plot real observations vs surrogate function
def plot(X, y, model):
    # scatter plot of inputs and real objective function
    pyplot.scatter(X, y)
    # line plot of surrogate function across domain
    Xsamples = asarray(arange(0, 1, 0.001))
    Xsamples = Xsamples.reshape(len(Xsamples), 1)
    ysamples, _ = surrogate(model, Xsamples)
    pyplot.plot(Xsamples, ysamples)
    # show the plot
    pyplot.show()
    

if test:
    # sample the domain sparsely with noise
    X = random(100)
    y = asarray([objective(x) for x in X])
    # reshape into rows and cols
    X = X.reshape(len(X), 1)
    y = y.reshape(len(y), 1)
    # define the model
    model = GaussianProcessRegressor()
    # fit the model
    model.fit(X, y)
    # plot before hand
    plot(X, y, model)
    # perform the optimization process
    for i in range(100):
        # select the next point to sample
        x = opt_acquisition(X, y, model)
        # sample the point
        actual = objective(x)
        # summarize the finding
        est, _ = surrogate(model, [[x]])
        print('>x=%.3f, f()=%3f, actual=%.3f' % (x, est, actual))
        # add the data to the dataset
        X = vstack((X, [[x]]))
        y = vstack((y, [[actual]]))
        # update the model
        model.fit(X, y)

    # plot all samples and the final surrogate function
    plot(X, y, model)
    # best result
    ix = argmax(y)
    print('Best Result: x=%.3f, y=%.3f' % (X[ix], y[ix]))

### sobol sampling plus Nelder-Mead

### simulated anealing

In [55]:
from scipy.interpolate import interp1d


# objective function
class tstFxn:
    def __init__(self):
        self.X = np.linspace(0,1,1000)
        self._y = np.asarray([self.objective(x) for x in self.X])
        self.f =  interp1d(self.X, self._y,kind='cubic')
        
    def objective(self,x, noise=0.1):
        noise = normal(loc=0, scale=noise)
        return (x**2 * sin(5 * pi * x)**6.0) + noise
    
    

    
#simulated anealing algorithm must have the following functions: 
# the feasable state space
# the Energetic function (which is the objective function) E()
# the candidate generator procedure (neighbor(x))
#        -generation of random candidate vector, with variable magnitude?
#         aniostropic exploration is better!!!
#thoughts for how I might solve this problem - 
#         boundry check or only sampling within the boundry seems a good strategy. 
#        -moving in one direction at at time. 

# acceptance probability function P()
# anealing schedule - number, number of anealing cycles, cooling rate
# some termination condition - total execution time or number of interations
# presensce of and number of restarts, deterministic or otherwise? 

#of these, cooling schedule and neighbor seem the most important functions


class SA:
    def __init__(self,boxConstraints):
        self.nIter = 1000;
        self.T = 1             #temperature variable on [1-0)
        self.N = 1             #neighborhood size coefficient [1-0)
        self.k = 1             #boltsman constant 
        self.C_half = 200      #half-life cooling              [trials]
        self.N_half = 150      #half-life of neighborhood size [trials]
        bcs = boxConstraints
        self.lMax = bcs[0,:]   #upper limits on box constraints for x variable
        self.lMin = bcs[1,:]   #lower limits on box constraints for x variable
    
    def sample(self):
        """
        sample the space for the next sample based on the neighborhood
        function
        input:
            x
        output:
            x_new
        """
        
        
    def optimize(self,obj):
        x0 = .5 ; fx0 = obj(x0)
        for n in range(self.nIter):
            x = np.random.random()
            fx = obj(x)
            
            #if better, keep
            if fx > fx0:
                fx0 = fx ; x0 = x
                
            #probablastically accept worse answer
            else:
                Δf = fx - fx0
                r = np.random.random()
                if r > np.exp(-Δf/self.k*self.T):
                    fx0 = fx ; x0 = x
                else:
                    pass
            
            #decrease temperature and neighborhood size
            self.T = .5**(n/self.C_half)
            self.N = .5**(n/self.N_half)
        
        return x0,fx0
                    
 



#if test:
#     #plot
#     tester = tstFxn()
#     sa = SA()

#     sa.optimize(tester.f)
    
#     %timeit sa.optimize(test.f)




